# AIChE 2018 Hands-On Tutorial: Building Biomolecular Models

**[Tucker Burgin](mailto:tburgin@umich.edu) and [Heather Mayes](mailto:hbmayes@umich.edu)**

*University of Michigan, Department of Chemical Engineering*

---

This is a Python 3 Jupyter notebook to guide you through an introduction to biomolecular modeling with OpenMM. This tutorial will begin with installation of the necessary software (including Conda) and then proceed into an example of constructing a protein model from a crystal structure and running a simple simulation with it. It is written for an audience that is entirely new to biomolecular modeling and simulation; however, although the python programming language will be used, this is not a python tutorial. Fortunately, only very minimal programming knowledge should be required!

## Installing Conda and creating a Python environment

We will use the Conda python interpreter because of its built in package management features. Specifically, we will use miniconda, which is a light-weight version. Complete install instructions can be found on the documentation website [here](https://conda.io/docs/user-guide/install/index.html), but in general simply follow [this](https://conda.io/miniconda.html) link, download the Python 3.7 installer appropriate for your OS, and execute it.

Once conda is installed, create a new Python environment using the following terminal commands (they won't execute here; copy and paste them into your own terminal):

You may need to give permission to install dependencies while executing the first line. These commands create and activate a new environment called "openmm_env", which means that you can install Python packages here without needing to worry about how they might interact with any other Python packages you might have on your system. One handy feature of Conda environments is that they belong to the user account that created them, which means you can use this method to install and use Python packages on systems where you don't have root access, such as on a high-performance computing cluster.

## Installing OpenMM and dependencies

Now we can install our needed software packages. Since we installed Conda, this is very straightforward:

Conda will automatically determine the necessary dependencies for the openmm package and install, update, and downgrade elements of the active environment to match. You can double-check that your installation was successful with the command:

This should produce output something like this:

From here you are ready to work with OpenMM. However, if you want the code in this Jupyter notebook to be executable, there's one more step to take: simply close the notebook, `source activate openmm_env` in the desired terminal instance, and then reopen it with `jupyter notebook <path/to/this/notebook>` to load it with your new environment. Alternatively, you can copy-and-paste the python code segments into a file on your computer and execute the code from there using whatever interpreter you like.

## Building a simple molecular model

Now we are ready to start working on our molecular model. First, we need to understand: what is a "model" in this context? That is, what information constitutes a complete molecular model, and how is that information encoded in the computer?

Generally, a molecular model consists of two types of files: topology and coordinates. Let's begin with coordinates, which consist very simply of a long list of coordinates in three-space (usually X, Y, and Z) indicating the positions of every atom in the system. Generally, a coordinate file contains no information about *what* each atom is; that's the role of a topology file! Topology files (also sometimes called 'top', parameter, parameter/topology, or 'prmtop' files, among other names) are paired with coordinate files, and encode the "special sauce" of chemistry that translates a list of coordinates into a model system. These files contain information like atom types, bond topology, residue names, and so on. More on this a little later.

For now, just remember that a complete model requires *both* of these files, as a pair, neither of which is very useful without the other. So how do we build them?

### Importing a crystal structure

Most molecular modeling of proteins, especially large ones, takes advantage of a crystal structure as a starting point. Crystal structures provide a very good approximation of the native folding state of a protein, without which we would need to somehow fold the protein ourselves! For this tutorial, we will use a very simple protein: the HIV gp120 C5 domain ([Guilhaudis *et al.* (2002) Euro. J. Biochem. 269(19):4860](http://doi.org/10.1046/j.1432-1033.2002.03187.x)). This is actually a small part of a larger protein, but this domain has been implicated in the process by which HIV infects host cells, and is therefore of interest as a drug target.

<img src="https://cdn.rcsb.org/images/rutgers/me/1meq/1meq.pdb-500.jpg" alt="PDB ID: 1MEQ" width="300"/>

So let's write some code to import this structure! First, we need to import some dependencies. These statements will appear as the first thing in any OpenMM script:

In [ ]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

Then, we need to download the crystal structure. This particular structure is available on the Protein Data Bank (PDB) with the code: 1MEQ. Go to [this](http://www.rcsb.org/structure/1MEQ) page, click the blue "download files" link on the top-right, and then click "PDB Format" from the drop-down menu. After your browser finishes downloading the file, move it to the same directory that this IPython notebook is located in. If you're not sure where that is, run this code:

In [ ]:
import os
os.path.realpath('./')

Alternatively, if you're using a separate interpreter, put it in the same directory as your .py file. Once the .pdb file is in the right place, we can read it into OpenMM like this:

In [ ]:
pdb = PDBFile('1meq.pdb')

If you get an error here, make sure that the file is in the right directory, that OpenMM is installed correctly, and that you ran the import statements above.

Now we are ready to work with our .pdb file! The .pdb format is a little bit special: it contains a mix of coordinate and topology information, which is possible because of the limited number of residue topologies that are found in proteins. The interested reader is encouraged to open up a .pdb file with a text editor in order to get a feel for the encoded information! This information will be useful in the next step...

### Building the model and the topology and coordinate files

Molecular models are really just that: models. They do not encode the entirety of physics from subatomic particles up (although some methods, called *ab initio*, do make an attempt). Instead, they apply approximations of the interactions between atoms in the form of what's called a "force field". There are lots of force fields, all with their own strengths and weaknesses, but in modeling proteins the two most popular are called Amber and CHARMM, respectively. Incidentally, these are also the names of molecular modeling programs (just like OpenMM), but more on that later. These force fields are popular because they have been exhaustively developed and tested specifically with biomolecules in mind. OpenMM comes with both readily available, so let's apply the Amber force field as such:

In [ ]:
forcefield = ForceField('amber14-all.xml', 'amber14/tip3p.xml')

This loads the 2014 release of the Amber forcefield, which contains parameterizations of protein residues (as well as nucleotides and many lipids and sugars, incidentally) as well as a the TIP3P water model. With a force field in hand, we can start building the completed model starting from the .pdb file.

When we downloaded the crystal structure from the PDB, we got the positions of all the atoms in the protein. But real proteins don't exist in a vacuum; they are solvated in water. In order to recreate this in our model, we need to create an OpenMM "Modeller" object:

In [ ]:
modeller = Modeller(pdb.topology, pdb.positions)

This is a tool that lets us modify the structure we imported from the .pdb file. Let's add a 5x5x5 nm cube of water around the crystal structure to represent the bulk solvent:

In [ ]:
modeller.addSolvent(forcefield, boxSize=Vec3(5.0, 5.0, 5.0)*nanometers, model='tip3p')

Of course, that 5 nm cube of water shouldn't be in a vacuum, either! To solve this issue, molecular models will often apply what's called a "box" (see the "boxSize" keyword argument above?) to the model. The box tells the model to substitute an "image" of itself at the boundary whenever a calculation crosses outside the box. If you've ever played the arcade game Pac-Man, this concept should feel familiar!

![Pac-Man](pacman.gif)

---

**A brief aside: adding hydrogens**

When you download a .pdb file representing a crystal structure, typically you will find that it is missing hydrogen atoms. This is because hydrogen is too small to show up reliably in X-ray crystallography. Generally this is no problem, as we can use basic chemical knowledge to add hydrogens wherever they're missing. However, in proteins there are some cases where it is not obvious whether or not a hydrogen is missing: titratible residues. These are amino acids whose side chains' protonation states depend on the pH. In OpenMM, we can add these hydrogens (and all missing hydrogens, in fact) at a given pH:

(Note that this code is not executable because we don't actually want to run it here. Unlike most .pdb files, PDB ID: 1MEQ actually *does* have hydrogens, so we skip this step). Unfortunately, there is another subtlety here in that the pKa of a residue actually depends on the local environment (*e.g.*, other nearby residues), so to handle this issue properly we need more advanced tools, such as [H++](http://biophysics.cs.vt.edu/). Again, though, this is not necessary in this case!

---

Once the waters and hydrogens have been added, the last step before we can perform a simulation is to collapse all this information into a single "system" object:

In [ ]:
# Make a System object
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, nonbondedCutoff=8*angstrom, constraints=HBonds)
# Save the Modeller object to a file so that we can run future simulations with it without rebuilding it
PDBFile.writeFile(modeller.topology, modeller.positions, open('output.pdb', 'w'))

There are a few keyword arguments to point out here. `nonbondedMethod` refers to the method used to calculate interaction potentials between atoms that are not covalently bonded. The method we chose here, `PME`, stands for Particle Mesh Ewald, and is considered quite standard. A related concept, the `nonbondedCutoff` indicates the distance beyond which the system will ignore (not bother to calculate) interaction potentials. This is fundamentally a time-saving approximation, but it is generally agreed-upon that interactions beyond about 8 Å do not meaningfully influence simulations. Finally, for the `constraints` keyword, we set `HBonds`. This setting tells the software to fix the bondlengths of all covalent hydrogen bonds. Without this setting, covalent hydrogen bonds vibrate rapidly and make calculations slower, but fixing them is unlikely to significantly change the energetics of the system, so this is a good opportunity for time-saving!

At this point, we can load the .pdb file we just wrote in any molecular visualization software to take a look at it. Here's a peek at what that might look like in the visualization program VMD:

![Rot](rot.gif)

A keen eye will notice some green dots floating around in the solvent. These are chlorine ions that OpenMM automatically added for us when solvating in order to equalize the overall charge of the system. Very helpful!

The System object formalism is unique to OpenMM; many other modeling programs (such as Amber or CHARMM, for instance) operate on topology and coordinate files directly, but when we use OpenMM to build the model in the first place it already has that information in-memory, so there's no reason to write out new files. However, if at this stage we want to try using a different modeling software, OpenMM can plug into some other software to produce these files (which we can then use as inputs for other programs). For example, the following code block produces Amber-format topology and coordinate files by plugging into the Amber program ParmEd **(skip to the next section if you're not interested in migrating to Amber just now)**:

In [ ]:
import parmed
structure = parmed.openmm.topsystem.load_topology(modeller.topology, system, modeller.positions)
bond_type = parmed.BondType(1.0, 1.0, list=structure.bond_types) # Necessary because of a discrepancy in how OpenMM and ParmEd handle "none" bonds. Not a particularly satisfactory solution, work in progress.
structure.bond_types.append(bond_type)
for bond in structure.bonds:
    if bond.type is None:
        bond.type = bond_type
structure.save('system.prmtop', overwrite=True) # Warning: will be several GB's in size.
structure.save('system.rst7', format='rst7', overwrite=True) # Note that this tries to write velocities, but there are none; to fix this, just delete the second number in the second line of the resulting file.

Note that this code may take some time to complete, so only run it if you actually want the output files! Also, you'll notice that this code uses another module, parmed. This can be easily installed in our Conda environment just like we installed OpenMM:

But again, don't worry about this unless you want to move over to Amber.

### Performing a simple simulation 

Once we've built a System object, we're ready to set up and run the simulations. First, we set up an Integrator object and plug in into our system:

In [ ]:
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)

As the name suggests, an Integrator is responsible for integrating the equations of motion (here we will use Langevin dynamics, the details of which are outside the scope of this tutorial) and obtaining the velocity vectors for every atom at every step. The step size is set to 2 fs here, and the temperature is 300 K, which are both very typical values. We also created a Simulation object as a function of our modeller topology, our System object, and the Integrator object that we just created. The last step is to set the positions of the atoms (also taken from the Modeller object) and then:

In [ ]:
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('traj.pdb', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

The first line tells OpenMM to optimize the structure to a local energy minimum. This removes any errors associated with the modelling process and makes simulations more convergent. The "reporters" steps just tell the program what to do with the results of the upcoming simulation -- the first line indicates that the coordines should be written to 'traj.pdb" every 100 steps, while the second one outputs simulation progress data to the console as it runs. Finally, the 'step' function runs the simulation for 10,000 steps.

It's that easy! After this step completes, you will find the output trajectory "traj.pdb" in your working directory. This file can be loaded into a visualization program to watch the trajectory. You are encouraged to try this yourself to explore your simulation and all the different ways you might visualize and analyze it ([here](https://www.ks.uiuc.edu/Development/Download/download.cgi?PackageName=VMD) is the VMD download page!), but here's what I got:

![Sim](sim.gif)

Note: You may notice if you visualize your own trajectory that the protein appears to have moved to the corner of the water box. This has to do with the box images we discussed earlier and is not an error.

## Conclusion

Molecular simulation is a rich and valuable field that benefits from the combined efforts of researchers from all over the world, all with their own particular applications and work flows. Due to time constraints, we had to leave a lot out of this tutorial, but here is a short list of topics for further reading should you be interested in builidng a larger or more sophisticated model:
* Disulphide bonds in proteins
* Choosing your water model (implicit or explicit) based on the needs of your simulations (*e.g.*, optimized for polarizability, self-diffusion coefficient, solvation energy, simulation speed, *etc.*)
* Coarse-graining
* Parameterization of non-protein ligands and substrates
* Different simulation ensembles
* Restraints and constraints

We hope that this tutorial was illustrative of what basic biomolecular modeling looks like and that it helps you to conceptualize what molecular simulations do -- and maybe, what they can do for your work!